In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Input, concatenate
import pickle
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import numpy as np
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)+1e-10, axis=-1))
earlyStopping=EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')

ckpt_path = './log/weights-{val_loss:.4f}.hdf5'
ckpt = ModelCheckpoint(ckpt_path,
                       monitor='val_loss', 
                       verbose=1, 
                       save_best_only=True, 
                       mode='min')
def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
#     line['is_peek'] = line['is_peek'].astype(str)
    line['is_crowd'] = line['is_crowd'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['hour_crowd'] = line['hour'] + line['is_crowd']
    
    line['hour'] = line['hour'].astype(int)
#     line['is_peek'] = line['is_peek'].astype(int)
    line['is_crowd'] = line['is_crowd'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['hour_crowd'] = line['hour_crowd'].astype(int)
    return line

with open('./data/train-id3-crowd-grid2.txt', 'rb') as data_file:
    train = pickle.load(data_file)
    #训练验证都划分
    #900 29 26 #800 27.9 25.9 #700 27.5 25.3 #27.4 24.8 #26.4 24.08 #23 21 #(90)250 21 19.5 #(80)17. 16.5
    #训练划分 验证不划分
    #250
train = train[(train['Diff_Time']<600)]
train = zuhe(train)
with open('./data/test-id3-crowd-grid2.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)

from sklearn import preprocessing
x11 = train['ID'].as_matrix().reshape([-1,1])
x12 = train['ID2'].as_matrix().reshape([-1,1])
# x13 = train['e_ij'].as_matrix().reshape([-1,1])

N1 = x11.shape[0]

x21 = test['ID'].as_matrix().reshape([-1,1])
x22 = test['ID2'].as_matrix().reshape([-1,1])
# x23 = test['e_ij'].as_matrix().reshape([-1,1])

N2 = x21.shape[0]

x1 = np.concatenate((x11,x21))
x1 = preprocessing.LabelEncoder().fit_transform(x1) #13963746x21567
print(x1.shape)#(14454126, 21567)
train['new_ID'] = x1[:N1]
test['new_ID'] = x1[N1:]

x2 = np.concatenate((x12,x22))
x2 = preprocessing.LabelEncoder().fit_transform(x2) #13963746x21567
print(x2.shape)#(14454126, 21567)
train['new_ID2'] = x2[:N1]
test['new_ID2'] = x2[N1:]

# x3 = np.concatenate((x13,x23))
# x3 = preprocessing.LabelEncoder().fit_transform(x3) #13963746x21567
# print(x3.shape)#(14454126, 21567)
# train['new_e_ID'] = x3[:N1]
# test['new_e_ID'] = x3[N1:]


/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(14223933,)
(14223933,)


In [4]:
#np.unique(x2).shape
from sklearn.utils import shuffle
line = shuffle(train)
train_num = int(0.9 * line.shape[0])
train = line[:train_num]
#!
dev = line[train_num:]
print(train.shape)
print(dev.shape)

(12360197, 35)
(1373356, 35)


In [ ]:
from keras.layers import Reshape, Conv2D, MaxPooling2D
col1 = ['new_ID']
col2 = ['new_ID2']
col3 = [c for c in train if
       c not in ['ID2', 'new_ID2','Unnamed: 0','is_crowd', 's_ij', 'e_ij','s_x', 's_y', 'e_x', 'e_y','new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time']]
#+gps 2 epoch 31
X_train1 = train[col1].values
X_train2 = train[col2].values
X_train3 = train[col3].values

y_train = train['Diff_Time'].values
print(col1,col2,col3)
print(X_train1.shape, X_train2.shape)
X_dev1 = dev[col1].values
X_dev2 = dev[col2].values
X_dev3 = dev[col3].values

y_dev = dev['Diff_Time'].values
# print(X_dev.shape, y_dev.shape)

input1 = Input(shape=(1,))
input2 = Input(shape=(1,))
input3 = Input(shape=(15,))

x1 = Embedding(21449, 256, input_length=1)(input1)
x1 = Flatten()(x1)
x2 = Embedding(7278, 256, input_length=1)(input2)
x2 = Flatten()(x2)
x3 = Dense(units=512)(input3)
x3 = Activation("relu")(x3)

x = concatenate([x1, x2, x3])
#CNN
x = Reshape((32, 32, 1))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(1, activation=None)(x)
model = Model(inputs=[input1, input2, input3], outputs=out)
print(model.summary())
# model.load_weights('./log/weights-21.7671.hdf5')
model.compile(loss=root_mean_squared_error, optimizer='adam', metrics=[root_mean_squared_error])
model.fit([X_train1,X_train2,X_train3], y_train, validation_data=([X_dev1,X_dev2,X_dev3], y_dev), epochs=40, batch_size=1024,callbacks=[ckpt,earlyStopping])
# model.save("keras3.h5")

['new_ID'] ['new_ID2'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'hour_weekday', 'hour_crowd']
(12360197, 1) (12360197, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 15)           0                                            
___________________________________________________________________________________________

In [ ]:
col1 = ['new_ID']
col2 = ['new_ID2']
col3 = [c for c in test if
       c not in ['new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
print(col1,col2,col3)
X_test1 = test[col1].values
X_test2 = test[col2].values t7
X_test3 = test[col3].values
print(X_test1.shape,X_test2.shape,X_test3.shape)

model.load_weights('./log/weights-21.7671.hdf5') #(21.7671==<0.03789)
y_pred1 =  model.predict([X_test1,X_test2,X_test3],batch_size=1024)[:,0]

test = test[['ID', 'new_dist',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday','is_crowd', 's_ij', 'e_ij','peak_workday']]
test.columns=['ID', 'Distance',  'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance1', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO', 
       'hour_weekday','is_crowd', 's_ij', 'e_ij','peak_workday']

test['pred1'] = y_pred1
test['pred2'] = y_pred1

sub1 = test[['O_LINENO','TERMINALNO', 'O_UP','Source_Station','Target_Station','O_TIME','pred1','pred2','Distance','Distance1']]
sub1.columns = ['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','pred1','pred2','Distance','Distance1']
sub1 = sub1.reset_index()
del sub1['index']

sub=pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
sub['realTime'] = pd.to_datetime(sub['realTime'],format='%Y-%m-%d %H:%M:%S')
sub2 = sub[['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime','distance']]
sub2=pd.merge(sub2,sub1,on=['LINE','TERMINALNO','UP','pred_start_stop_ID','pred_end_stop_ID','realTime'],how='left')
sub2